# Initialization

In [ ]:
from google.colab import drive
drive.mount("drive")

Mounted at drive


In [ ]:
!pip install transformers==4.0.0 vncorenlp fairseq==0.10.0

     |████████████████████████████████| 1.4MB 5.7MB/s 
     |████████████████████████████████| 2.7MB 19.4MB/s 
     |████████████████████████████████| 1.7MB 29.6MB/s 
     |████████████████████████████████| 2.9MB 31.3MB/s 
     |████████████████████████████████| 890kB 47.4MB/s 
     |████████████████████████████████| 122kB 12.2MB/s 
     |████████████████████████████████| 71kB 8.6MB/s 
     |████████████████████████████████| 112kB 51.5MB/s 
     |████████████████████████████████| 276kB 46.2MB/s 
  Created wheel for vncorenlp: filename=vncorenlp-1.0.3-cp36-none-any.whl size=2645936 sha256=f9cbfae8748a97151c3af0fedf0558dc88a54caa22edbe0b2151c71daab42bbc
  Stored in directory: /root/.cache/pip/wheels/09/54/8b/043667de6091d06a381d7745f44174504a9a4a56ecc9380c54
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp36-none-any.whl size=893261 sha256=c3aa43f7638f418470a35f7a66082481f1ce3177c790d3dc8db613025f49546a
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31

In [ ]:
%cd drive/My Drive/Colab Notebooks/chatbot

/content/drive/My Drive/Colab Notebooks/chatbot


In [ ]:
from model import Net
from data_loader import *
from functions import *

Special tokens have been added in the vocabulary, make sure the associated word embedding are fine-tuned or trained.


In [ ]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
max_tokens_length = 256
 
class Args():
  def __init__(self):
    self.max_turn = 2
    self.over_sample = False # turn 1 + turn 2
    self.normalize = True   # True: target = dot - norm | False: target = dot product
    self.prepend = True    # emo prepend
    self.stop_crit_num_epochs = 3 # early stop
    self.learning_rate = 1e-5
    self.epochs = 30
    self.hits_at_nb_cands = 100 # p@1,100
    self.display_iter = 100 # help="Frequency of train logging"
    self.batch_size = 32
    self.optimizer = "adamax"
    self.train = False  # Train or evaluate
    self.log_file = "logs/phobert_prepend_normalize_finetune_bosung.txt"
    self.model_file = "models/phobert_prepend_normalize_finetune_bosung.pt"

option = Args()
logger = get_logger(option)

12/31/2020 02:34:30 PM: [ CONFIG:
{
    "batch_size": 32,
    "display_iter": 100,
    "epochs": 30,
    "hits_at_nb_cands": 100,
    "learning_rate": 1e-05,
    "log_file": "logs/phobert_prepend_normalize_finetune_bosung.txt",
    "max_turn": 2,
    "model_file": "models/phobert_prepend_normalize_finetune_bosung.pt",
    "normalize": true,
    "optimizer": "adamax",
    "over_sample": false,
    "prepend": true,
    "stop_crit_num_epochs": 3,
    "train": false
} ]


# Load dataset

In [ ]:
train_dataset = EmpDataset(
  "ED/train_manual - bosung",
#   "ED/train_manual",
#   "ED/train_auto",
#   "ED/train_manual_compound",
  maxlen = max_tokens_length,
  history_len = option.max_turn,
  prepend = option.prepend,
  over_sample = option.train and option.over_sample
)

train_iter = DataLoader(
  dataset     = train_dataset,
  batch_size  = option.batch_size,
  shuffle     = option.train,
  num_workers = 0,
  collate_fn  = batchify,
  pin_memory  = True,
)

100%|██████████| 24295/24295 [01:15<00:00, 322.71it/s]


In [ ]:
dev_dataset = EmpDataset(
#   "ED/valid_auto",
  "ED/valid_manual",
#   "ED/valid_manual_compound",
  maxlen = max_tokens_length,
  history_len = option.max_turn,
  prepend = option.prepend,
  over_sample = option.train and option.over_sample
)

dev_iter = DataLoader(
  dataset     = dev_dataset,
  batch_size  = option.batch_size,
  shuffle     = option.train,
  num_workers = 0,
  collate_fn  = batchify,
  pin_memory  = True,
)

100%|██████████| 2496/2496 [00:07<00:00, 325.35it/s]


In [ ]:
torch.save(train_iter,"torch_pre_load/train_manual_noprepend.pth")
# torch.save(dev_iter,"torch_pre_load/dev_manual.pth")

In [ ]:
train_iter = torch.load("torch_pre_load/train_auto.pth")
# dev_iter = torch.load("torch_pre_load/dev_auto.pth")

# Train

In [ ]:
net = Net(device)

if device == "cuda":
  torch.cuda.set_device(-1) # get the lastest device (GPU)
  net = torch.nn.DataParallel(net)
  net.cuda()

In [ ]:
###############
### Train #####
###############
    
if option.optimizer == "adamax":
    lr = option.learning_rate
    named_params_to_optimize = filter(
        lambda p: p[1].requires_grad, net.named_parameters()
    )
    params_to_optimize = (p[1] for p in named_params_to_optimize)
    optimizer = optim.Adamax(params_to_optimize, lr=lr)

    
start_time = time.time()
#best_loss = float("+inf")

with torch.no_grad():
    best_loss = validate(
        0,
        net,
        dev_iter,
        nb_candidates=option.hits_at_nb_cands,
        normalize = option.normalize
    )

 
for epoch in range(0, option.epochs):
    train(epoch, start_time, net, optimizer, option, train_iter, normalize = option.normalize)
    with torch.no_grad():
        loss = validate(
            epoch,
            net,
            dev_iter,
            nb_candidates=option.hits_at_nb_cands,
            normalize = option.normalize
        )
        if loss < best_loss:
            best_loss = loss
            best_loss_epoch = epoch
            logging.info(f"New best loss, saving model to {option.model_file}")
            torch.save(net.state_dict(), f"{option.model_file}")
        # Stop if it's been too many epochs since the loss has decreased
        if option.stop_crit_num_epochs != -1:
            if epoch - best_loss_epoch >= option.stop_crit_num_epochs:
                break

12/30/2020 03:11:37 PM: [ Processing candidate top-K ]
12/30/2020 03:11:37 PM: [ Valid (shuffled): Epoch = 0 | avg loss = 5.365 | batch P@1 = 3.94 % | P@1,100 = 1.36% | P@3,100 = 4.36% | P@10,100 = 13.73% | valid time = 5.13 (s) ]
12/30/2020 03:12:26 PM: [ train: Epoch = 0 | iter = 100/363 | loss = 3.616 | batch P@1 = 4.44 % | elapsed time = 54.90 (s) ]
12/30/2020 03:13:17 PM: [ train: Epoch = 0 | iter = 200/363 | loss = 3.369 | batch P@1 = 6.52 % | elapsed time = 105.74 (s) ]
12/30/2020 03:14:08 PM: [ train: Epoch = 0 | iter = 300/363 | loss = 2.592 | batch P@1 = 15.03 % | elapsed time = 156.95 (s) ]
12/30/2020 03:14:40 PM: [ train: Epoch = 0 | iter = 363/363 | loss = 2.354 | batch P@1 = 19.02 % | elapsed time = 188.27 (s) ]
12/30/2020 03:14:40 PM: [ train: Epoch 0 done. Time for epoch = 183.06 (s) ]
12/30/2020 03:14:45 PM: [ Processing candidate top-K ]
12/30/2020 03:14:45 PM: [ Valid (shuffled): Epoch = 0 | avg loss = 2.472 | batch P@1 = 33.33 % | P@1,100 = 20.45% | P@3,100 = 35.36%

# Kết quả



- finetune UVM old:
 + P@1,100 = 20.67%
 + BLEU = 7.08

- prepend + finetune + UVM old
 + BLEU = 6.74
 + P@1,100 = 23.33% | P@3,100 = 50.00% | P@10,100 = 72.33%

- normalize + finetune + UVM old
 + BLEU = 6.70
 +  P@1,100 = 21.33% | P@3,100 = 43.67% | P@10,100 = 66.00%


- Gốc, turn = 2
 + P@1,100 = 25.00%
 + BLEU = 6.19

- Finetune + normalize:
 + P@1,100 = 27.00% | P@3,100 = 51.33% | P@10,100 = 77.00%
 + BLEU = 6.38%

- Emo prepend, lọc ? !, turn = 2
 + P@1,100 = 28.33% | P@3,100 = 57.67% | P@10,100 = 82.00%
 + BLEU = 6.55

- Emo prepend, turn = 4, lọc ? !, cosine - norm
 + P@1,100 = 31.33% | P@3,100 = 62.33% | P@10,100 = 85.00%
 + BLEU = 6.545

- finetune, Emo prepend, normalize:
 + P@1,100 = 30.67% | P@3,100 = 59.33% | P@10,100 = 85.33%
 + BLEU = 6.46
 
- pretrain + Fine tuned, emo prepend, oversample, lọc ?!, dot - norm
 + P@1,100 = 37.67% | P@3,100 = 74.00% | P@10,100 = 91.67%
 + BLEU: 6.51%

- pretrain + Fine tuned, emo prepend, oversample, lọc ?!, dot - norm, evaluate turn = 1
 + P@1,100 = 41.33% | P@3,100 = 66.67% | P@10,100 = 84.33%
 + BLEU: 6.22%
 
- pretrain + Fine tuned, emo prepend, oversample, lọc ?!
 + P@1,100 = 38.67% | P@3,100 = 77.67% | P@10,100 = 92.67%
 + BLEU = 5.82%

- pretrain UVA, emotion prepend, normalize
 + P@1,100 = 36.67% | P@3,100 = 73.33% | P@10,100 = 92.33%
 + BLEU = 6.42

- pretrain, finetuned bosung, emotion prepend, normalize
 + P@1,100 = 37.67% | P@3,100 = 72.00% | P@10,100 = 93.00%
 + BLEU = 6.37%
- finetuned bosung, emotion prepend, normalize
 + P@1,100 = 31.33% | P@3,100 = 62.67% | P@10,100 = 85.67%
 + BLEU = 6.23

# Demo

In [ ]:
net = Net(device, False)

if device == "cuda":
  torch.cuda.set_device(-1) # get the lastest device (GPU)
  net = torch.nn.DataParallel(net)
  net.cuda()
  
net.load_state_dict(torch.load(option.model_file), strict = False)
# net.load_state_dict(torch.load("models/phobert_auto.pt"), strict = False)
net.eval()
 

In [ ]:
import time
start_time = time.time()

all_cands = []

with torch.no_grad():
    for i, ex in enumerate(train_iter):
        batch_size = ex[0].size(0)
        params = [
            field
            if field is not None
            else None
            for field in ex
        ]
            
        cands = net(params[1][:,0,:])
        all_cands.append(cands)

    all_cands = torch.cat(all_cands, dim = 0)


end_time = time.time()
print("Total time load candidates: ", end_time - start_time)

Total time load candidates:  15.854732751846313


In [ ]:
torch.save(all_cands, "torch_pre_load/all_cands_prepend_normalize_manual_bosung.pth")

In [ ]:
# all_cands = torch.load("torch_pre_load/all_cands_finetuned_normalize.pth")

In [ ]:
def predict(context, top_n=5, normalize=True):
    """
    returns a list of top_n tuples ("sentence", "score")
    """
  
    all_input_ids = txt2vec(context, 256)
    with torch.no_grad():
        if device == "cuda":
            all_input_ids = all_input_ids.cuda(non_blocking=True)
       
        ctx = net(all_input_ids)
        scores, index = score_candidates(ctx, all_cands, top_n, normalize)
        response = []
        for i, (score, index) in enumerate(zip(scores.squeeze(0), index.squeeze(0)), 1):
            response.append((train_dataset[index], float(score)))
     
        return response

In [ ]:
sentence = "bà tôi vừa qua đời"
outs = predict(sentence, top_n = 10, normalize = option.normalize)
for item in outs:
    print("Score: ", item[1], "\nResponse: ",  str.format("{}",tokenizer.decode(item[0][1][0][2:-1]).replace("_", " ")))

Score:  7.895349979400635 
Response:  tôi rất tiếc khi nghe điều đó. bà tôi cũng mất gần đây.
Score:  7.838269233703613 
Response:  rất tiếc khi biết điều đó. bây giờ tôi chỉ còn lại mình bà thôi. ông tôi đã qua 1 tháng trước
Score:  7.703534126281738 
Response:  tôi rất tiếc khi nghe điều đó. bạn đã dành nhiều thời gian bên bà không?
Score:  7.218474864959717 
Response:  buồn vậy. bà mất vì bệnh hay vì tuổi cao?
Score:  7.198681831359863 
Response:  tôi bà đã không còn sống với bạn nữa?
Score:  7.1826653480529785 
Response:  tôi rất tiếc khi nghe điều đó. tôi hy vọng bà ấy sẽ sớm bình phục.
Score:  7.066975116729736 
Response:  tôi rất tiếc khi nghe điều đó. bà mới qua đời gần đây sao?
Score:  7.053089141845703 
Response:  gần đây tôi đã mất bà tôi. tôi nghĩ rằng tôi sẽ không vượt qua được.
Score:  6.938544273376465 
Response:  điều đó thật tệ. tôi yêu mẹ tôi rất nhiều, tôi không thể tưởng tượng được rằng mẹ đã mất...
Score:  6.854318618774414 
Response:  tôi xin chia buồn cùng anh, t

In [ ]:
index = 230

print("Sentence: ", tokenizer.decode(train_dataset[index][0][0]))
print("Target: ", tokenizer.decode(train_dataset[index][1][0]))

print(10*"*")

outs = predict(tokenizer.decode(train_dataset[index][0][0]), 10, normalize = option.normalize)
for item in outs:
    print("Score: ", item[1], "\nResponse: ",  str.format("{}", tokenizer.decode(item[0][1][0][2:-1])))

Sentence:  <s> tôi đang cắt cỏ trong vườn, có cục đá bay vào nhà hàng_xóm làm bễ kiếng của sổ của họ. </s>
Target:  <s> 6 ôi không … rồi có chuyện gì không? </s>
**********
Score:  7.315635681152344 
Response:  ôi, nghe thật kinh_khủng! nó có phải đồ_hoạ không?
Score:  7.296921730041504 
Response:  bất_ngờ nha. một cái lỗ dưới hàng_rào ư? tôi rất tiếc khi nghe điều đó.
Score:  7.146152019500732 
Response:  haha. đôi_khi nó trông có_vẻ như_vậy đấy. chúng có_thể đậu ở những nơi khuất mãi_mãi và chỉ di_chuyển vào giấy cuối_cùng khi bạn vung cái vợt đập ruồi thôi. gioongs như seal team six hay_là những thứ giống vậy.
Score:  6.83128547668457 
Response:  lấy chai xịt hot shots hoặc raid spray, phun từ khoảng 6 đến 9 mét vào không_trung. tôi phun tất_cả mọi nơi từ mặt_đất đến mái nhà của tôi. phun và chạy. lol
Score:  6.7333197593688965 
Response:  nghe có_vẻ đau_đớn. tôi sẽ đổ rác trước toà thị_chính.
Score:  6.729208469390869 
Response:  trời! em ghét chuyện giống như_vậy và em cá là nó là

# Evaluate for test set

## P@1,100

In [ ]:
test_dataset = EmpDataset(
#   "ED/test_manual_compound",
  "ED/test_manual",
  maxlen = max_tokens_length,
  history_len = option.max_turn,
  prepend = option.prepend,
  over_sample = False
)

sh_test_iter = DataLoader(
  dataset     = test_dataset,
  batch_size  = option.batch_size,
  shuffle     = True,
  num_workers = 0,
  collate_fn  = batchify,
  pin_memory  = True,
)

un_test_iter = DataLoader(
  dataset     = test_dataset,
  batch_size  = option.batch_size,
  shuffle     = False,
  num_workers = 0,
  collate_fn  = batchify,
  pin_memory  = True,
)

100%|██████████| 694/694 [00:02<00:00, 336.11it/s]


In [ ]:
with torch.no_grad():
  validate(
    0,
    net,
    sh_test_iter,
    shuffle = True,
    nb_candidates = option.hits_at_nb_cands,
    normalize = option.normalize
  )

with torch.no_grad():
  validate(
    0,
    net,
    un_test_iter,
    shuffle = False, 
    nb_candidates= option.hits_at_nb_cands,
  )

12/31/2020 02:45:14 AM: [ Processing candidate top-K ]
12/31/2020 02:45:14 AM: [ Valid (True): Epoch = 0 | avg loss = 1.458 | batch P@1 = 58.46 % | P@1,100 = 40.67% | P@3,100 = 61.00% | P@10,100 = 85.67% | valid time = 1.48 (s) ]
12/31/2020 02:45:16 AM: [ Processing candidate top-K ]
12/31/2020 02:45:16 AM: [ Valid (False): Epoch = 0 | avg loss = 2.375 | batch P@1 = 40.31 % | P@1,100 = 31.67% | P@3,100 = 63.00% | P@10,100 = 85.67% | valid time = 1.39 (s) ]


## BLEU

In [ ]:
# Lọc 3 token ở đầu thừa trước khi tính BLEU
filter = 2 # <s> and <emo>

for i in range(len(test_dataset)):
    test_dataset[i][1] = test_dataset[i][1][:,filter:]

un_test_iter = DataLoader(
  dataset     = test_dataset,
  batch_size  = option.batch_size,
  shuffle     = False,
  num_workers = 0,
  collate_fn  = batchify,
  pin_memory  = True,
)

In [ ]:
# from fairseq import bleu # version 0.9.0
from fairseq.scoring import bleu # version 0.10.0

def BLEU_calculate(data_loader, top_n=5, normalize=True):
    scorer = bleu.Scorer(pad_idx, tokenizer.eos_token_id, tokenizer.unk_token_id)
    with torch.no_grad():
        for i, ex in enumerate(data_loader):
            batch_size = ex[0].size(0)
            params = [
                field
                if field is not None
                else None
                for field in ex
            ]
    
            ctx = net(params[0][:,0,:])
            cands =   params[1][:,0,:]
             
            _, index = score_candidates(ctx, all_cands, top_n, normalize)
            index = index[:,0].cpu()
            predicts = []
         
            #BLEU calculate
            for i in range(batch_size):
              scorer.add(
                  cands[i].type(torch.IntTensor),  
                  train_dataset[index[i]][1][:,filter:].type(torch.IntTensor)
              )

    def get_float(result):
      return float(result.split(',')[0].split("=")[-1])

    return (
        get_float(scorer.result_string(order=4))+
        get_float(scorer.result_string(order=3))+
        get_float(scorer.result_string(order=2))+
        get_float(scorer.result_string(order=1))
    )/4

print(str.format("BLEU: {:.2f}%", BLEU_calculate(un_test_iter, normalize = option.normalize)))

BLEU: 6.22%


# test

In [ ]:
pip install gradio

In [ ]:
def clean_answer(outs):
  rs = []
  for item in outs:
    rs.append(tokenizer.decode(item[0][1][0][2:-1]).replace("_", " "))
  return "\n".join(rs)

def greet(sentence):
  outs = predict(net, sentence, 2)
  return clean_answer(outs)

 

mặt tốt là, nó sẽ có thể là một lý do tuyệt vời để đặt bánh pizza.
đó chính là tất cả những gì về lễ tạ ơn lol


In [ ]:
import gradio as gr

def clean_answer(outs):
  rs = []
  for item in outs:
    rs.append(tokenizer.decode(item[0][1][0][3:-1]).replace("_", " "))
  return "\n".join(rs)

def greet(sentence):
  outs = predict(net, sentence, 1)
  return clean_answer(outs)

iface = gr.Interface(fn=greet, inputs="text", outputs="text")
iface.launch(debug=True, share=True)

In [ ]:
with torch.no_grad():
  validate(
    0,
    net,
    dev_iter,
    shuffle = False, 
    nb_candidates= option.hits_at_nb_cands,
  )

11/12/2020 03:09:28 AM: [ Processing candidate top-K ]
11/12/2020 03:09:28 AM: [ Valid (False): Epoch = 0 | avg loss = 2.830 | batch P@1 = 34.62 % | P@1,100 = 26.27% | P@3,100 = 57.18% | P@10,100 = 79.91% | valid time = 4.87 (s) ]


In [ ]:
!nvidia-smi

Sun Nov 15 12:59:39 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 455.32.00    Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   41C    P0    33W / 250W |  16277MiB / 16280MiB |      0%      Default |
|                               |                      |                 ERR! |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

# Googletrans

In [ ]:
!pip install googletrans==3.1.0a0

  Created wheel for googletrans: filename=googletrans-3.1.0a0-cp36-none-any.whl size=16368 sha256=d3645b15f823da2ec4e870f1c081c480414f97bfec52165c4f3d4c7c0251b20b
  Stored in directory: /root/.cache/pip/wheels/27/7a/a0/aff3babbb775549ce6813cb8fa7ff3c0848c4dc62c20f8fdac
Successfully built googletrans
  Found existing installation: googletrans 3.0.0
    Uninstalling googletrans-3.0.0:
      Successfully uninstalled googletrans-3.0.0


In [ ]:
from googletrans import Translator
translator = Translator()

translator.translate(translator.translate('mẹ thấy con tệ quá', dest='en').text, dest="vi").text


'Tôi nghĩ bạn thật tệ'

In [ ]:

class TextData(data.Dataset):
    def __init__(
        self,
        splitname,
        maxlen=256,  # max number of tokens per sentence
        history_len=2,
        prepend=True,  # add label as prefix
        over_sample=False
    ):

        df = open(f"{splitname}.csv", "r", encoding="utf-8").readlines()
        self.max_hist_len = history_len * 2 - 1
        self.data = []
        history = []
        for i in tqdm(range(1, len(df))):

            cparts = df[i - 1].strip().split(",")
            sparts = df[i].strip().split(",")
            if cparts[0] == sparts[0]:

                history.append(preprocess(cparts[5]))
                idx = int(sparts[1])
                if (idx % 2) == 0:
                    # Lay cac turn cuoi
                    # SOC start of comment
                    sentence2 = preprocess(sparts[5])

                    if over_sample:
                        history[-1] = history[-1].replace(
                            "?", "").replace("!", "").strip()
                        # if(len(history) == 1):
                        self.data.append([history[-1], sentence2, sparts[2]])
                        sentence1 = " | ".join(history[-self.max_hist_len:])
                        if(len(history) != 1):
                            self.data.append([sentence1, sentence2, sparts[2]])

                    else:
                        history[-1] = history[-1].replace(
                            "?", "").replace("!", "").strip()
                        sentence1 = " | ".join(history[-self.max_hist_len:])
                        self.data.append([sentence1, sentence2, sparts[2]])
            else:
                history = []

    def __len__(self):
        return len(self.data)

    def __getitem__(self, index):
        return self.data[index]


test_dataset = TextData(
    "ED/test_manual",
    maxlen=max_tokens_length,
    history_len=option.max_turn,
    prepend=option.prepend,
    over_sample=option.train and option.over_sample
)

100%|██████████| 694/694 [00:00<00:00, 60648.96it/s]


In [ ]:
# excited, scared, angry, sad, neutral
all_emotions_cls = [
    [
        "confident",
        "impressed",
        "joyful",
        "faithful",
        "excited",
        "grateful",
        "prepared",
        "hopeful",
        "proud",
        "content",
        "surprised",
        "caring",
        "trusting",
        "anticipating",
    ],
    [
        "afraid",
        "ashamed",
        "anxious",
        "guilty",
        "apprehensive",
        "embarrassed",
    ],
    [
        "jealous",
        "disgusted",
        "angry",
        "annoyed",
        "furious",
    ],
    [
        "disappointed",
        "sad",
        "lonely",
        "devastated",
        "terrified",
    ],
    [
        "sentimental",
        "nostalgic"
    ]
]

In [ ]:

# excited, scared, angry, sad, neutral
def check_emo(emo):
    if emo in all_emotions_cls[0]:
        return "excited"

    if emo in all_emotions_cls[1]:
        return "scared" 

    if emo in all_emotions_cls[2]:
        return "angry" 
    if emo in all_emotions_cls[3]:
        return "sad" 
    if emo in all_emotions_cls[4]:
        return "neutral" 

In [ ]:
with open("danhgia_duytridoithoai.csv", "w", encoding="utf-8") as f:
    f.write("Câu gốc, Ground truth, Dự đoán\n")
    for i in test_dataset:
        # print(i[0], "\nground truth: ", i[2])
        outs = predict(i[0], top_n = 1, normalize = option.normalize)
        item = outs[0]
        indx = int(tokenizer.decode(item[0][1][0][1:2]))
        f.write(str.format("{},{},{}\n", i[0].replace(",", "_comma_"), check_emo(i[2]), check_emo(lbEnc.classes_[indx])))




In [ ]:
lbEnc.classes_

array(['afraid', 'angry', 'annoyed', 'anticipating', 'anxious',
       'apprehensive', 'ashamed', 'caring', 'confident', 'content',
       'devastated', 'disappointed', 'disgusted', 'embarrassed',
       'excited', 'faithful', 'furious', 'grateful', 'guilty', 'hopeful',
       'impressed', 'jealous', 'joyful', 'lonely', 'nostalgic',
       'prepared', 'proud', 'sad', 'sentimental', 'surprised',
       'terrified', 'trusting'], dtype='<U12')

In [ ]:
lbEnc.transform(lbEnc.classes_)

array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16,
       17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31])

In [ ]:
i = test_dataset[5]
print(i[0], "\nground truth: ", i[2])
outs = predict(i[0], top_n = 10, normalize = option.normalize)
item = outs[0]
indx = int(tokenizer.decode(item[0][1][0][1:2]))
print("out: ", lbEnc.classes_[indx])